In [0]:
import pandas as pd
import numpy as np
from sklearn import svm
import os

In [2]:
df =  pd.read_csv('./email_body.csv')
df.head()

,Unnamed: 0,Subject,Content Type,Content
0,0,=?ISO3166-1?B?bWl0a8OkIG92YXQgc3V1bm5pdGVsbWFz...,text/html,Etsitkö kuumia tyttöjä ja naisia?\n\n\nHaluatk...
1,1,=?ISO3166-1?B?T2xlbiBrYWRvbm51dA==?=,text/html,Etsitkö kuumia tyttöjä ja naisia?\n\n\nHaluatk...
2,2,Pharmcy: ask my wife,text/plain,b'Refill it: http://www542.Laurihirvonen.xn--c...
3,3,Pharmcy: ask my wife,text/html,b'Refill it: www542.Laurihirvonen.xn--c1aj7aam...
4,4,[SPAM: 5.842] Tilisi on vahingoittunut,text/plain,b'Hei!\n\nTämä on sinulle tärkeää tietoa!\n\nM...


In [24]:
df["Subject"].value_counts().head()

Get Paid $5900 Daily With UNDERGROUND Profit Opportunity    476
Undelivered Mail Returned to Sender                         219
Married wives want sex                                      140
Best seller                                                 136
Cure High Cholesterol By Cutting Out This ONE Ingredient     80
Name: Subject, dtype: int64

In [27]:
df[df["Subject"].str.contains("Re: ", na=False)]["Subject"].value_counts().head() # Mark as spam for subject with Re: only

Re: Hello dear,                                                            6
Re: Your trial set offer                                                   6
Re: Best World web-drug-shop                                               3
Re:Re: Aubine We do guarntee highh-quailty medications, instat devliery    2
Re: Meeting                                                                2
Name: Subject, dtype: int64

In [28]:
df[df["Subject"].str.contains("SPAM: ", na=False)]["Subject"].value_counts().head() # Mark as spam for subject contain SPAM:

[SPAM: 5.021] Juha, saat luottoja jopa 250 000 kruunuun ja muokkaa\n\tunelmiesi loma todelliseksi.                                                                                27
[SPAM: 3.512]\n\t=?UTF-8?Q?T=C3=A4yt=C3=A4_lainahakemus_-_Saatlainap=C3=A4=C3=A4t=C3=B6kse?=\n\t=?UTF-8?Q?n?= heti.                                                                6
[SPAM: 5.009] Sinun ei tarvitse ostaa Bitcoinia tai\n\t=?UTF-8?Q?tehd=C3=A4_suuriasijoituksia,_jotta_p=C3=A4=C3=A4sisit_hy=C3=B6?=\n\t=?UTF-8?Q?tym=C3=A4=C3=A4n_siit=C3=A4.?=     6
[SPAM: 5.011] Tarjoamme todellisen kokemuksen\n\tkryptovaluuttamarkkinoilla.                                                                                                       6
[SPAM: 5.021] =?UTF-8?Q?T=C3=A4ss=C3=A4?= on mahdollisuus ansaita                                                                                                                  5
Name: Subject, dtype: int64

In [29]:
df[df["Subject"].str.contains("@", na=False)]["Subject"].value_counts().head()

tommy.watt@bbbs.net Your Dish washer Repairs are Covered.                         4
please be my f@ck buddy                                                           4
[SPAM: 4.101] Lainaa 100 - 50000 euroa heti tilille\n\tjuha.oravala@d-fence.fi    2
tommy.watt@bbbs.net Your Plumbing Repairs are Covered.                            2
[SPAM: 5.01] Saat kasinovoittosi 5 minuutissa juha.oravala@d-fence.fi             2
Name: Subject, dtype: int64

In [30]:
df[df["Subject"].str.contains("\\$", na=False)]["Subject"].value_counts().head()

Get Paid $5900 Daily With UNDERGROUND Profit Opportunity             476
Goldman Sachs Forecast Places Bitcoin at $13791 in the Short-term     68
$25 per hour jobs on Facebook, now hiring!                            28
URGENT TRANSFER OF YOUR US$4,500,000.00                                9
Flag this: Custom Canvas starting at $4.90 | 93% OFF!                  8
Name: Subject, dtype: int64

In [0]:
subjects = df["Subject"].values.astype(str)
subject_list = df["Subject"].values.astype(str).tolist()

In [4]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
token = word_tokenize(' '.join(subject_list))

In [0]:
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

In [0]:
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

In [0]:
def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [0]:
import re
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [0]:
# stemmed_words = stem_words(token)
# lemmatize_words = lemmatize_verbs(stemmed_words)
token = to_lowercase(token)
normalize_words = remove_stopwords(remove_punctuation(token))

In [11]:
df_normalize_words = pd.DataFrame(normalize_words, columns=["word"])
pd.set_option("display.max_rows", None)
df_normalize_words["word"].value_counts().head(n=20)

b              3346
utf8           2300
iso31661       1917
q              1196
get             927
free            611
profit          562
daily           521
paid            492
opportunity     487
5900            478
underground     478
spam            427
want            402
today           369
new             357
home            349
best            336
meet            304
one             298
Name: word, dtype: int64

In [12]:
!pip install urlextract

In [13]:
!pip install urlextract
from urlextract import URLExtract
from urllib.parse import urlparse

extractor = URLExtract()
urls = []
domains = []
for msg in df["Content"].values:
  for url in extractor.gen_urls(str(msg)):
      urls.append(url)
for url in urls:
  domain = urlparse(url).netloc
  domains.append(domain)
domain_df = pd.DataFrame({"domain": domains})
domain_df["domain"].value_counts()

hotevelyn.su                                         1250
                                                      868
bethany.su                                            476
hotalexis.su                                          469
arianna.su                                            342
jemma.su                                              338
layla.su                                              334
laura.su                                              313
jenna.su                                              255
violeta.su                                            240
sweetrebecca.su                                       238
meetmargo.su                                          233
milanda.su                                            227
toppharmacy24.su                                      206
alicia.su                                             184
toppharmacy365.su                                     151
meetanna.su                                           114
fonts.googleap

In [14]:
!pip install urlextract
from urlextract import URLExtract
from urllib.parse import urlparse

extractor = URLExtract()
urls = []
domains = []
for msg in df["Subject"].values:
  for url in extractor.gen_urls(str(msg)):
      urls.append(url)
url_df = pd.DataFrame({"url": urls})
url_df["url"].value_counts()

sikkertlan.dk             6
Call247Legal.com          4
21.com                    3
valtavalaina.fi           3
VAEligibility.us          2
bbbs.net                  1
d-fence.eu                1
roskapostitorjunta.com    1
Name: url, dtype: int64

In [0]:
clf = svm.OneClassSVM(gamma='auto')

In [16]:
!pip3 install --quiet tensorflow
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
import tensorflow as tf
import tensorflow_hub as hub

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/3

In [18]:
os.environ["TFHUB_CACHE_DIR"] = '/tmp'
embed = hub.load(module_url)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  subjects_embeddings = session.run(embed(subjects))

In [22]:
train_data = subjects_embeddings

{'outputs': array([[-0.02187547,  0.00563569,  0.06801578, ..., -0.06945018,
          0.06385136, -0.05918045],
        [ 0.08281939, -0.0339064 ,  0.0288986 , ..., -0.02597575,
          0.0386723 , -0.1052671 ],
        [ 0.03479153,  0.0408328 , -0.00406672, ...,  0.03207142,
          0.05884926, -0.01002041],
        ...,
        [-0.03498279, -0.07389237,  0.03231907, ..., -0.0490117 ,
         -0.04115262,  0.00882119],
        [ 0.01447839, -0.07957227, -0.00838248, ..., -0.01858078,
         -0.00054261,  0.04289978],
        [-0.04654759, -0.00833349,  0.00813342, ...,  0.00382909,
          0.05883846, -0.03994447]], dtype=float32)}

In [23]:
clf.fit(train_data['outputs'])

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
            max_iter=-1, nu=0.5, random_state=None, shrinking=True, tol=0.001,
            verbose=False)

In [26]:
clf.predict(train_data['outputs'])

array([ 1,  1,  1, ..., -1, -1,  1])